In [4]:
from curriculum import build_curriculum_graph
from students import simulate_students, get_eligible_courses
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

# 1. Build Curriculum Graph
print("Building curriculum graph...")
graph = build_curriculum_graph()

# 2. Simulate 100 Students
print("Simulating students...")
students = simulate_students(graph)

# 3. Save and Preview
df = pd.DataFrame(students)
df.to_csv("simulated_student_data.csv", index=False)
print(df.head())

# 4. Visualize Graph
plt.figure(figsize=(10, 8))
areas = nx.get_node_attributes(graph, 'area')
color_map = {'Core': 'skyblue', 'AI': 'lightgreen', 'Security': 'salmon', 'Data Science': 'khaki'}
colors = [color_map.get(areas[n], 'gray') for n in graph.nodes()]

pos = nx.spring_layout(graph, k=1.5)
nx.draw(graph, pos, with_labels=True, node_color=colors, node_size=3000,
        font_size=10, font_weight='bold', arrowsize=20)
plt.title("Curriculum Prerequisite Graph")
plt.savefig("curriculum_graph_visualization.png")

Building curriculum graph...
Simulating students...


PermissionError: [Errno 13] Permission denied: 'simulated_student_data.csv'

In [5]:
# --- STEP 1: Function to score eligible courses based on interest + graph position ---
def score_course(course_id, student, graph):
    score = 0
    course_area = graph.nodes[course_id].get('area', '')
    
    # +1 if course matches interest
    if course_area in student['interests']:
        score += 1

    # +2 if it unlocks more advanced courses (many outgoing edges)
    outgoing = list(graph.successors(course_id))
    score += len(outgoing) * 0.5  # each unlock gives +0.5

    return score

# --- STEP 2: Recommend Courses for a Student ---
def recommend_courses(student, graph, max_courses=5):
    eligible = get_eligible_courses(graph, student['completed_courses'], student['failed_courses'])
    if not eligible:
        return []

    # Score eligible courses
    scored_courses = [(course, score_course(course, student, graph)) for course in eligible]
    scored_courses.sort(key=lambda x: x[1], reverse=True)  # sort by score descending

    # Recommend top N
    recommended = [course for course, score in scored_courses[:max_courses]]
    return recommended

# --- STEP 3: Try it for 10 students and print results ---
print("\nTop course recommendations for 10 students:\n" + "-"*50)
for student in students[:10]:
    recommendations = recommend_courses(student, graph)
    print(f"Student {student['student_id']}")
    print(f" Interests: {student['interests']}")
    print(f" Completed: {list(student['completed_courses'].keys())}")
    print(f" Recommended: {recommendations}\n")



Top course recommendations for 10 students:
--------------------------------------------------
Student 0
 Interests: ['Security']
 Completed: ['MATH101', 'CS102', 'CS101', 'DS301', 'SEC301', 'AI301', 'SEC401', 'AI401']
 Recommended: []

Student 1
 Interests: ['AI']
 Completed: ['CS101', 'CS102', 'MATH101', 'AI301', 'DS301', 'SEC301', 'SEC401', 'AI401']
 Recommended: []

Student 2
 Interests: ['AI', 'Data Science']
 Completed: ['CS102', 'MATH101', 'CS101']
 Recommended: ['AI301', 'DS301', 'SEC301']

Student 3
 Interests: ['AI']
 Completed: ['CS101', 'CS102', 'MATH101']
 Recommended: ['AI301', 'SEC301', 'DS301']

Student 4
 Interests: ['Security']
 Completed: ['MATH101', 'CS101', 'CS102', 'DS301', 'SEC301', 'AI301', 'AI401', 'SEC401']
 Recommended: []

Student 5
 Interests: ['Data Science', 'AI']
 Completed: ['CS101', 'CS102', 'MATH101', 'DS301', 'AI301', 'SEC301', 'SEC401', 'AI401']
 Recommended: []

Student 6
 Interests: ['Security', 'AI']
 Completed: ['MATH101', 'CS101', 'CS102', 'AI

In [6]:
import numpy as np
class AdvisingEnvironment:
    def __init__(self, graph, student):
        self.graph = graph
        self.student = student
        self.reset()

    def reset(self):
        self.completed = set(self.student['completed_courses'].keys())
        self.failed = set(self.student['failed_courses'])
        self.interests = self.student['interests']
        self.term = 0
        self.history = []
        return self.get_state()

    def get_state(self):
        return tuple(sorted(self.completed)), self.term

    def get_eligible_courses(self):
        return get_eligible_courses(self.graph, dict.fromkeys(self.completed, 3.0), self.failed)

    def step(self, selected_courses):
        reward = 0
        new_courses = []
        for course in selected_courses:
            grade = np.round(np.random.normal(3.0, 0.6), 2)  # simulate grade
            if grade >= 2.0:
                self.completed.add(course)
                if course in self.failed:
                    self.failed.remove(course)
                reward += grade
                if self.graph.nodes[course]['area'] in self.interests:
                    reward += 1  # bonus for interest alignment
            else:
                self.failed.add(course)
                reward -= 1  # penalty for failing

            new_courses.append((course, grade))

        self.term += 1
        self.history.append(new_courses)
        return self.get_state(), reward


In [13]:
from collections import defaultdict
import random
class QLearningAgent:
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.2):
        self.Q = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def select_action(self, env):
        eligible = env.get_eligible_courses()
        if not eligible:
            return []

        if random.random() < self.epsilon:
            return random.sample(eligible, k=min(len(eligible), 3))

        state = env.get_state()
        q_scores = {}
        for course in eligible:
            q_scores[course] = self.Q[state][course]
        
        top_courses = sorted(q_scores.items(), key=lambda x: x[1], reverse=True)[:3]
        return [course for course, _ in top_courses]

    def update(self, state, action, reward, next_state):
        for course in action:
            q_sa = self.Q[state][course]
            max_q_next = max(self.Q[next_state].values() or [0])
            self.Q[state][course] = q_sa + self.alpha * (reward + self.gamma * max_q_next - q_sa)

In [11]:
def train_agent(graph, students, episodes=50):
    agent = QLearningAgent()
    for _ in range(episodes):
        for student in students:
            env = AdvisingEnvironment(graph, student)
            state = env.reset()

            for _ in range(5):  # simulate up to 5 terms
                action = agent.select_action(env)
                next_state, reward = env.step(action)
                agent.update(state, action, reward, next_state)
                state = next_state
    return agent

In [14]:
print("\n🎓 RL-Based Recommendations for 10 Students\n" + "-"*50)
agent = train_agent(graph, students)

for student in students[:10]:
    env = AdvisingEnvironment(graph, student)
    recommended = agent.select_action(env)
    print(f"Student {student['student_id']}")
    print(f" Interests: {student['interests']}")
    print(f" Completed: {list(student['completed_courses'].keys())}")
    print(f" RL Recommended: {recommended}\n")


🎓 RL-Based Recommendations for 10 Students
--------------------------------------------------
Student 0
 Interests: ['Security']
 Completed: ['MATH101', 'CS102', 'CS101', 'DS301', 'SEC301', 'AI301', 'SEC401', 'AI401']
 RL Recommended: []

Student 1
 Interests: ['AI']
 Completed: ['CS101', 'CS102', 'MATH101', 'AI301', 'DS301', 'SEC301', 'SEC401', 'AI401']
 RL Recommended: []

Student 2
 Interests: ['AI', 'Data Science']
 Completed: ['CS102', 'MATH101', 'CS101']
 RL Recommended: ['DS301', 'SEC301', 'AI301']

Student 3
 Interests: ['AI']
 Completed: ['CS101', 'CS102', 'MATH101']
 RL Recommended: ['DS301', 'SEC301', 'AI301']

Student 4
 Interests: ['Security']
 Completed: ['MATH101', 'CS101', 'CS102', 'DS301', 'SEC301', 'AI301', 'AI401', 'SEC401']
 RL Recommended: []

Student 5
 Interests: ['Data Science', 'AI']
 Completed: ['CS101', 'CS102', 'MATH101', 'DS301', 'AI301', 'SEC301', 'SEC401', 'AI401']
 RL Recommended: []

Student 6
 Interests: ['Security', 'AI']
 Completed: ['MATH101', 'CS1